# Обучение с подкреплением

***Второй этап, обучение на победу***

**Выполнил Кузин Мирослав**

In [87]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [88]:
import os

work_dir = "./drive/MyDrive/Colab Notebooks/ml4course_1laba/"

from google.colab import files
src = list(files.upload().values())[0]
open('mancala.py','wb').write(src)


Saving mongala.py to mongala (1).py


5349

In [2]:
from mancala import Kalah
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np
import pandas as pd
import time
import random

**Болваны**

In [55]:
# Бот простой, рандомно выбирает не нулевые элементы
def do_simple_bot_step(state: torch.Tensor) -> int:
    nonzero_state_indexs = torch.nonzero(state[:6]).flatten()
    rez = nonzero_state_indexs[torch.randint(0, len(nonzero_state_indexs), (1,))[0]]
    return rez + 1


# Бот простой, рандомно выбирает не нулевые элементы
def do_prof_bot_step(state: torch.Tensor) -> int:
    nonzero_state_indexs = torch.nonzero(state[:6]).flatten()
    state_copy = state.clone()
    captured = []
    for cursor in nonzero_state_indexs:
        n = int(cursor)
        # print(state_copy)
        temp = int(state_copy[cursor])
        state_copy[cursor] = 0
        while temp > 0:
            temp -= 1
            if cursor in range(0, 6):
                cursor += 1
            elif cursor == 6:
                cursor = 13
            elif cursor in range(9, 14):
                cursor -= 1
            elif cursor == 8:
                cursor = 0

            state_copy[cursor] += 1

        if cursor < 6 and state_copy[cursor] == 1 and state_copy[cursor%6-6] != 0:
            state_copy[6] += sum([state_copy[cursor], state_copy[cursor%6-6]])
            captured.append((n, sum([state_copy[cursor], state_copy[cursor%6-6]])))
            state_copy[cursor], state_copy[cursor%6-6] = 0., 0.
        state_copy = state.clone()

    if captured != []:
        rez_ind = sorted(captured, key=lambda x: x[1])
        rez = rez_ind[-1][0]
    else:
        rez = nonzero_state_indexs[torch.randint(0, len(nonzero_state_indexs), (1,))[0]]
    return rez + 1


do_simple_bot_step(torch.Tensor([1, 0, 1, 0, 0, 0, 0]))
do_prof_bot_step(torch.Tensor([1, 0, 1, 0, 1, 0, 0, 0, 2, 7, 3, 4, 2, 13]))

5

**Модель**

In [91]:
# model = torch.load(work_dir + "model2_winner_ver1.pt")
model = torch.nn.Sequential(
    torch.nn.Linear(14, 28),
    torch.nn.ReLU(),
    torch.nn.Linear(28, 84),
    torch.nn.ReLU(),
    torch.nn.Linear(84, 6),
    torch.nn.Softmax(dim = -1)
)


**Настройка обучения**

In [145]:
def loss_func(probs: torch.Tensor, action: int, m: Categorical, R: int, gamma: int):
    alpha = 0.9
    # print(Categorical(probs).log_prob(action))
    return -alpha*R*m.log_prob(action)

loss = loss_func
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-4, amsgrad=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


**Обучение**

In [146]:
# Проверка работы модели
game = Kalah()
model.to(device)
model.to(torch.double)

print(game.get_general_state())
model(game.get_general_state().to(torch.double))

tensor([6, 6, 6, 6, 6, 6, 0, 0, 6, 6, 6, 6, 6, 6])


tensor([1.2293e-04, 9.0163e-01, 9.8240e-02, 9.3740e-06, 4.4173e-07, 4.5774e-08],
       dtype=torch.float64, grad_fn=<SoftmaxBackward0>)

*Награды и штрафы*

In [152]:
# Второй этап обучения
winner_reward_stage_2 = 2.5
loser_reward_stage_2 = 0
draw_reward_stage_2 = 0
bad_step_reward_stage_2 = -25 # общий
good_step_reward_stage_2 = -2
good_step_captured_reward_stage_2 = -1

In [153]:
game = Kalah()
episodes_count = 10000
neuronet_walker_queue = 1
count_win_neuronet = 0
count_choisen_zero = 0
gamma = 2
loss_story = []
check_optim = False

for episode in range(0, episodes_count):
    game.set_new_game()
    tr = []
    if episode % 2:
        old_player_making_step = game.get_player_making_step()
        neuronet_walker_queue = 2
        while not game.get_game_over() and old_player_making_step == game.get_player_making_step():
            bot_action = do_simple_bot_step(game.get_state())
            rezult_step = game.take_step(bot_action)
            # print("bot took step", game.get_player_making_step(), "Был выбор", bot_action, rezult_step)
    else:
        neuronet_walker_queue = 1

    while not game.get_game_over():

        # Выбор хода
        probs = model(game.get_state().to(torch.double))
        m = Categorical(probs)
        action = m.sample()

        # Выбор награды и выполнение хода
        reward = 0
        old_state = game.get_state().to(torch.double)
        old_player_making_step = game.get_player_making_step()

        rezult_step = game.take_step(action + 1)
        # print("neuronet took step", game.get_player_making_step(), "Был выбор", action + 1, rezult_step)
        if rezult_step == "Куча пустая, выберите другую!":
            count_choisen_zero += 1
            reward = bad_step_reward_stage_2
        elif rezult_step == "Хороший ход!":
            reward = good_step_reward_stage_2
            bad_choise = []
        elif rezult_step == "Хороший ход! Захват!":
            reward = good_step_captured_reward_stage_2
            bad_choise = []

        tr.append([old_state, action, reward, m])

        while not game.get_game_over() and old_player_making_step != game.get_player_making_step():
            bot_action = do_simple_bot_step(game.get_state())
            rezult_step = game.take_step(bot_action)
            # print("bot took step", game.get_player_making_step(), "Был выбор", bot_action, rezult_step)

    rwrd = 0
    if game.get_winner() != None:
        if game.get_winner() != 0:
            if neuronet_walker_queue == game.get_winner():
                count_win_neuronet += 1
                rwrd += winner_reward_stage_2
            else:
                rwrd += loser_reward_stage_2
        else:
            rwrd += draw_reward_stage_2
    # tr[-1][-2] += rwrd if tr[-1][-2] > 0 else 0

    loss = 0.
    count_played_step = len(tr) # Кол-во сыгранных ходов
    for id_current_step in range(count_played_step):
        R = 0.
        for id_next_step in range(id_current_step, count_played_step):
            R += (gamma**(id_current_step-id_next_step))*(tr[id_next_step][2] + rwrd)
        loss += loss_func(model(tr[id_current_step][0]), tr[id_current_step][1], tr[id_current_step][3], R, gamma)

    if not check_optim and episode > 1000:
        check_optim = True
        for g in optimizer.param_groups:
            g['lr'] = 1.0e-5
        print("updated loss")

    if not episode % 200:
        print(loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

tensor(1.6262, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-10.0289, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(8.1681, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(1.8380, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-6.7433, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2.7991, dtype=torch.float64, grad_fn=<AddBackward0>)
updated loss
tensor(-0.5355, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2.8124, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2.0672, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(6.0972, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(5.0567, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-15.6638, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(-0.7612, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(5.0721, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2.7283, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(2.9472, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(5.6790, dtype

In [154]:
print(check_optim)
print(count_choisen_zero)

True
206


**Тестирование модели**

In [156]:
game = Kalah()
episodes_count = 10000
neuronet_walker_queue = 1
count_win_neuronet = 0
count_choisen_zero = 0
bad_choise = []
rezult_step = "Хороший ход!"

for episode in range(episodes_count):
    game.set_new_game()
    if episode % 2:
        old_player_making_step = game.get_player_making_step()
        neuronet_walker_queue = 2
        while not game.get_game_over() and old_player_making_step == game.get_player_making_step():
            bot_action = do_simple_bot_step(game.get_state())
            rezult_step = game.take_step(bot_action)
            # print("bot took step", game.get_player_making_step(), "Был выбор", bot_action, rezult_step)
    else:
        neuronet_walker_queue = 1

    while not game.get_game_over():

        # Выбор хода
        probs = model(game.get_state().to(torch.double))
        action = probs.argmax()

        while action in bad_choise:
            probs[probs.argmax()] = 0
            action = probs.argmax()

        # Выбор награды и выполнение хода
        reward = 0
        old_state = game.get_state().to(torch.double)
        old_player_making_step = game.get_player_making_step()

        rezult_step = game.take_step(action + 1)
        if rezult_step == "Куча пустая, выберите другую!":
            count_choisen_zero += 1
            # reward = bad_step_reward_stage_1
            print("Neuronet took step", game.get_player_making_step(), "Был выбор", action + 1, rezult_step)
            # print("state:")
            # game.print_state()
            print("Номер эпизода происшествия:", episode)
            bad_choise.append(action)
        else:
            bad_choise = []
        # elif rezult_step == "Хороший ход!":
        #     reward = good_step_reward_stage_1
        # elif rezult_step == "Хороший ход! Захват!":
        #     reward = good_step_captured_reward_stage_1

        while not game.get_game_over() and old_player_making_step != game.get_player_making_step():
            bot_action = do_simple_bot_step(game.get_state())
            rezult_step = game.take_step(bot_action)
            if rezult_step == "Куча пустая, выберите другую!":
                print("Bot took step", game.get_player_making_step(), "Был выбор", bot_action, rezult_step)


    if game.get_winner() != None:
        if game.get_winner() != 0:
            if neuronet_walker_queue == game.get_winner():
                count_win_neuronet += 1
                reward += winner_reward_stage_2
            else:
                reward += loser_reward_stage_2
        else:
            reward += draw_reward_stage_2

    # print("Очередь хода нейронки", neuronet_walker_queue)
    # print("Результат:", game.get_player_winner(), 'k', game.get_winner())
    # print("Номер попытки:", episode)
    # print("Награда/Штраф:", reward)

    # print(episode)

print(count_choisen_zero)
print(count_win_neuronet/episodes_count)

Neuronet took step 2 Был выбор tensor(2) Куча пустая, выберите другую!
Номер эпизода происшествия: 5223
1
0.7375


In [157]:
is_save = True
# is_save = False
if is_save:
    torch.save(model, work_dir+"model2_winner_top_ver2.pt")